In [1]:
from myml.nbinit import *

In [2]:
from etl.collection.explorer_migration import migrate_csv_explorer, _get_explorer_config, ExplorerMigration
from etl.paths import STEP_DIR
# from etl.collections.utils import extract_definitions, dump_yaml_with_anchors, move_field_to_top
from etl.config import OWIDEnv

# config = migrate_csv_explorer("ipcc-scenarios", owid_env=OWIDEnv.from_staging('master'))

owid_env = OWIDEnv.from_staging('master')

explorer_config = _get_explorer_config(owid_env, 'ipcc-scenarios')

for block in explorer_config['blocks']:
    if block['type'] == 'table':
        block['args'][0] = f'https://catalog.ourworldindata.org/explorers/emissions/latest/ipcc_scenarios/{block["args"][1]}.csv'

In [3]:
from etl.files import yaml_dump

# Create ExplorerMigration object
migration = ExplorerMigration(explorer_config, 'ipcc-scenarios')
config = migration.run()

# Some display names are wrong
for view in config['views']:
    y = view['indicators']['y'][0]
    table_name, ind_name = y['catalogPath'].split('#')

    # Fix forcing_aerosols
    if ind_name == 'forcing_aerosols':
        ind_name = 'forcing_aerosol'
        y['catalogPath'] = 'global#forcing_aerosol'
        y['display'] = {
            'name': 'Radiative forcing (aerosols)',
            'type': 'Numeric',
            'unit': 'watts per meter squared',
            'shortUnit': 'W/m²'
        }


    if 'display' in y:
        display_name = y['display']['name']
        if 'secondary_energy_' in ind_name and 'secondary_energy_elec' not in ind_name and 'electricity' in display_name:
            print(f"Fixing {ind_name} display name {display_name}")
            y['display']['name'] = y['display']['name'].replace('electricity', 'energy')

# There are two methanes
for dim in config['dimensions']:
    if dim['slug'] == 'sub_metric':
        dim['choices'] = [c for c in dim['choices'] if c['name'] != 'Methane (CH4)']

Fixing secondary_energy_heat display name Secondary electricity (heat)
Fixing secondary_energy_gas display name Secondary electricity (gas)
Fixing secondary_energy_oil display name Secondary electricity (oil)
Fixing secondary_energy_hydrogen display name Secondary electricity (hydrogen)
Fixing secondary_energy_liquid_biofuel display name Secondary electricity (biofuels)
Fixing secondary_energy_heat display name Secondary electricity (heat)
Fixing secondary_energy_gas display name Secondary electricity (gas)
Fixing secondary_energy_oil display name Secondary electricity (oil)
Fixing secondary_energy_hydrogen display name Secondary electricity (hydrogen)
Fixing secondary_energy_liquid_biofuel display name Secondary electricity (biofuels)
Fixing secondary_energy_heat display name Secondary electricity (heat)
Fixing secondary_energy_gas display name Secondary electricity (gas)
Fixing secondary_energy_oil display name Secondary electricity (oil)
Fixing secondary_energy_hydrogen display name

## Create ipcc_scenarios.meta.yml

In [4]:
definitions = {
    'common': {
        'display': {
            'entityAnnotationsMap': """
SSP1 - 1.9: Sustainability - Taking the Green Road
SSP1 - 2.6: Sustainability - Taking the Green Road
SSP1 - 3.4: Sustainability - Taking the Green Road
SSP1 - 4.5: Sustainability - Taking the Green Road
SSP1 - Baseline: Sustainability - Taking the Green Road
SSP2 - 1.9: Middle of the Road
SSP2 - 2.6: Middle of the Road
SSP2 - 3.4: Middle of the Road
SSP2 - 4.5: Middle of the Road
SSP2 - 6.0: Middle of the Road
SSP2 - Baseline: Middle of the Road
SSP3 - 3.4: Regional Rivalry - A Rocky Road
SSP3 - 4.5: Regional Rivalry - A Rocky Road
SSP3 - 6.0: Regional Rivalry - A Rocky Road
SSP3 - Baseline: Regional Rivalry - A Rocky Road
SSP4 - 2.6: Inequality - A Road Divided
SSP4 - 3.4: Inequality - A Road Divided
SSP4 - 4.5: Inequality - A Road Divided
SSP4 - 6.0: Inequality - A Road Divided
SSP4 - Baseline: Inequality - A Road Divided
SSP5 - 1.9: Fossil-fueled Development - Taking the Highway
SSP5 - 2.6: Fossil-fueled Development - Taking the Highway
SSP5 - 3.4: Fossil-fueled Development - Taking the Highway
SSP5 - 4.5: Fossil-fueled Development - Taking the Highway
SSP5 - 6.0: Fossil-fueled Development - Taking the Highway
SSP5 - Baseline: Fossil-fueled Development - Taking the Highway
    """.strip()
        }
    }
}

meta = {
    "definitions": definitions,
    "tables": {'ipcc_scenarios': {"variables": {}}},
}

for view in config['views']:
    y = view['indicators']['y'][0]
    table_name, ind_name = y['catalogPath'].split('#')

    if table_name != 'global':
        continue

    if 'display' in y:
        d = {
            "title": y['display']['name'],
            "unit": y['display']['unit'],
        }
        if 'shortUnit' in y['display']:
            d['short_unit'] = y['display']['shortUnit']
    else:
        d = {}

    meta['tables']['ipcc_scenarios']['variables'][ind_name] = d

# fix typos
# variables = meta['tables']['ipcc_scenarios']['variables']
# del variables['forcing_aerosols']
# variables['forcing_aerosol'] = {
#     "title": "Radiative forcing (aerosols)",
#     "unit": "watts per meter squared",
#     "short_unit": "W/m²",
# }


path = '/Users/mojmir/projects/etl/etl/steps/data/grapher/emissions/2022-08-30/ipcc_scenarios.meta.yml'
with open(path, 'w') as f:
    f.write(yaml_dump(meta))

## Create ipcc_scenarios.config.yml

In [5]:
from etl.collection.explorer_migration import reorder_dimensions

new_views = []

for view in config['views']:
    y = view['indicators']['y'][0]
    table_name, ind_name = y['catalogPath'].split('#')

    # Rename catalogPath - add region as a suffix
    y["catalogPath"] = f"ipcc_scenarios#{ind_name}__region_{view['dimensions']['region']}"

    # Delete fields coming from the dataset
    if 'display' in y:
        del y['display']['name']
        y['display'].pop('sourceLink', None)
        y['display'].pop('sourceName', None)
        y['display'].pop('dataPublishedBy', None)

        del y['display']['unit']
        y['display'].pop('shortUnit', None)
        y['display'].pop('transform', None)

        y["display"].pop("annotationsColumnSlug", None)

    del view['config']['sourceDesc']

    view['config'].pop('relatedQuestionUrl', None)
    view['config'].pop('relatedQuestionText', None)

    new_views.append(view)

config['views'] = new_views

config['config']['hideAnnotationFieldsInTitle'] = True

# Reorder dimensions
config = reorder_dimensions(config, ['metric', 'sub_metric', 'rate', 'region'])

# Dump explorer config
path_new = STEP_DIR / "export/explorers/emissions/latest/ipcc_scenarios.config.yml"
path_new.parent.mkdir(parents=True, exist_ok=True)
# Overwrite the file always
with open(path_new, "w") as f:
    f.write(yaml_dump(config))